In [ ]:
import numpy as np
from scipy.stats import norm

# Step 1: Data Generation
# Simulate 100 flips each for a fair coin (p=0.5) and a biased coin (p=0.8), 10 times each.
np.random.seed(42)  # For reproducibility
n_flips = 10
n_samples = 100
fair_coin_flips = np.random.binomial(n_flips, 0.5, n_samples)
biased_coin_flips = np.random.binomial(n_flips, 0.8, n_samples)
all_flips = np.concatenate((fair_coin_flips, biased_coin_flips))

# Step 2: Initialize Parameters
mu = np.array([5, 8])  # Initial means
sigma_squared = np.array([2.5, 2.5])  # Initial variances
pi = np.array([0.5, 0.5])  # Initial mixing coefficients

# Step 3 and 4: EM Algorithm
tolerance = 1e-6
max_iterations = 100
n_components = 2
n_iterations = 0
log_likelihood = 0

# EM Algorithm
for iteration in range(max_iterations):
    n_iterations += 1

    # E-step: Calculate responsibilities
    responsibilities = np.zeros((n_samples * 2, n_components))
    for k in range(n_components):
        responsibilities[:, k] = pi[k] * norm.pdf(all_flips, mu[k], np.sqrt(sigma_squared[k]))
    responsibilities /= responsibilities.sum(axis=1, keepdims=True)

    # M-step: Update parameters
    Nk = responsibilities.sum(axis=0)
    mu = np.sum(all_flips[:, np.newaxis] * responsibilities, axis=0) / Nk
    sigma_squared = np.sum(responsibilities * (all_flips[:, np.newaxis] - mu)**2, axis=0) / Nk
    pi = Nk / (n_samples * 2)

    # Check for convergence
    new_log_likelihood = np.sum(np.log(np.sum(responsibilities * pi, axis=1)))
    if np.abs(new_log_likelihood - log_likelihood) < tolerance:
        break
    log_likelihood = new_log_likelihood

# Final parameter values
final_parameters = {
    'means': mu,
    'variances': sigma_squared,
    'mixing_coefficients': pi,
    'log_likelihood': log_likelihood,
    'iterations': n_iterations
}

# Step 6: Assign Clusters
# Using the final parameters, calculate the maximum responsibility to assign clusters
cluster_assignments = np.argmax(responsibilities, axis=1)

final_parameters, cluster_assignments[:10]  # Show the first 10 assignments as an example


({'means': array([5.39011497, 8.68023882]),
  'variances': array([3.12379877, 0.56981736]),
  'mixing_coefficients': array([0.68545712, 0.31454288]),
  'log_likelihood': -119.97995259349143,
  'iterations': 100},
 array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0]))